In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/30 13:18:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

In [3]:
!hdfs dfs -cp holidays2.csv hdfs://nn:9000/holidays2.csv

In [4]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("hdfs://nn:9000/sf.csv"))

In [5]:
from pyspark.sql.functions import col, expr
cols = [col(c).alias(c.replace(" ", "_")) for c in df.columns]
df.select(cols).write.format("parquet").save("hdfs://nn:9000/sf.parquet")

23/10/30 13:20:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
!hdfs dfs -rm hdfs://nn:9000/sf.csv

Deleted hdfs://nn:9000/sf.csv


In [7]:
(spark.read
 .format("parquet")
 .load("hdfs://nn:9000/sf.parquet")
 .createOrReplaceTempView("calls")
)

In [10]:
(spark.read
 .format("csv")
 .option("inferSchema", True)
 .option("header", True)
 .load("hdfs://nn:9000/holidays2.csv")
 .createOrReplaceTempView("holidays")
)

In [11]:
spark.table("calls")

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [12]:
spark.table("holidays")

DataFrame[date: string, holiday: string]

In [ ]:
# can we pair up calls with holidays?

In [13]:
# SQL
spark.sql("""
SELECT *
FROM calls
INNER JOIN holidays ON calls.Call_Date = holidays.date
LIMIT 5
""").toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods,date,holiday
0,201461772,94,20060706,Medical Incident,05/25/2020,05/25/2020,05/25/2020 01:34:55 PM,05/25/2020 01:34:55 PM,05/25/2020 01:36:14 PM,05/25/2020 01:36:19 PM,...,MEDIC,1,10,10,Bayview Hunters Point,201461772-94,POINT (-122.38626545834089 37.73782577837169),1,05/25/2020,Memorial Day
1,201463874,E32,20060913,Medical Incident,05/25/2020,05/25/2020,05/25/2020 11:16:21 PM,05/25/2020 11:17:54 PM,05/25/2020 11:18:27 PM,05/25/2020 11:19:53 PM,...,ENGINE,1,9,11,Excelsior,201463874-E32,POINT (-122.42955213038633 37.729273863971024),7,05/25/2020,Memorial Day
2,201463019,T18,20060826,Medical Incident,05/25/2020,05/25/2020,05/25/2020 06:39:44 PM,05/25/2020 06:41:34 PM,05/25/2020 06:55:28 PM,05/25/2020 06:57:32 PM,...,TRUCK,2,8,7,Inner Sunset,201463019-T18,POINT (-122.46829612745225 37.751748874988515),14,05/25/2020,Memorial Day
3,201460900,E11,20060611,Medical Incident,05/25/2020,05/25/2020,05/25/2020 09:22:49 AM,05/25/2020 09:24:51 AM,05/25/2020 09:25:17 AM,05/25/2020 09:26:11 AM,...,ENGINE,1,6,8,Mission,201460900-E11,POINT (-122.42413130758577 37.752703146369065),20,05/25/2020,Memorial Day
4,201463227,B03,20060859,Alarms,05/25/2020,05/25/2020,05/25/2020 07:40:19 PM,05/25/2020 07:41:33 PM,05/25/2020 07:41:44 PM,05/25/2020 07:42:33 PM,...,CHIEF,2,3,6,Financial District/South Beach,201463227-B03,POINT (-122.40169229261626 37.78658885907634),8,05/25/2020,Memorial Day


In [14]:
calls = spark.table("calls")
holidays = spark.table("holidays")

In [17]:
calls["Call_Date"] == holidays["date"]

Column<'(Call_Date = date)'>

In [20]:
# same as SQL, but with DataFrames
calls.join(holidays, on=calls["Call_Date"] == holidays["date"], how="inner").limit(5).toPandas()

,Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,...,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,RowID,case_location,Analysis_Neighborhoods,date,holiday
0,201461772,94,20060706,Medical Incident,05/25/2020,05/25/2020,05/25/2020 01:34:55 PM,05/25/2020 01:34:55 PM,05/25/2020 01:36:14 PM,05/25/2020 01:36:19 PM,...,MEDIC,1,10,10,Bayview Hunters Point,201461772-94,POINT (-122.38626545834089 37.73782577837169),1,05/25/2020,Memorial Day
1,201463874,E32,20060913,Medical Incident,05/25/2020,05/25/2020,05/25/2020 11:16:21 PM,05/25/2020 11:17:54 PM,05/25/2020 11:18:27 PM,05/25/2020 11:19:53 PM,...,ENGINE,1,9,11,Excelsior,201463874-E32,POINT (-122.42955213038633 37.729273863971024),7,05/25/2020,Memorial Day
2,201463019,T18,20060826,Medical Incident,05/25/2020,05/25/2020,05/25/2020 06:39:44 PM,05/25/2020 06:41:34 PM,05/25/2020 06:55:28 PM,05/25/2020 06:57:32 PM,...,TRUCK,2,8,7,Inner Sunset,201463019-T18,POINT (-122.46829612745225 37.751748874988515),14,05/25/2020,Memorial Day
3,201460900,E11,20060611,Medical Incident,05/25/2020,05/25/2020,05/25/2020 09:22:49 AM,05/25/2020 09:24:51 AM,05/25/2020 09:25:17 AM,05/25/2020 09:26:11 AM,...,ENGINE,1,6,8,Mission,201460900-E11,POINT (-122.42413130758577 37.752703146369065),20,05/25/2020,Memorial Day
4,201463227,B03,20060859,Alarms,05/25/2020,05/25/2020,05/25/2020 07:40:19 PM,05/25/2020 07:41:33 PM,05/25/2020 07:41:44 PM,05/25/2020 07:42:33 PM,...,CHIEF,2,3,6,Financial District/South Beach,201463227-B03,POINT (-122.40169229261626 37.78658885907634),8,05/25/2020,Memorial Day


In [23]:
# how many calls on each kind of holiday?
(calls
 .join(holidays, on=calls["Call_Date"] == holidays["date"], how="inner")
 .groupby("holiday")
 .count()
 .orderBy("count")
 .toPandas())

,holiday,count
0,Inauguration Day,798
1,Juneteenth National Independence Day,1968
2,Christmas Day,8179
3,Thanksgiving Day,8186
4,Veterans Day,8674
5,Memorial Day,8830
6,Washington's Birthday,9064
7,"Birthday of Martin Luther King, Jr.",9557
8,Labor Day,9778
9,New Year's Day,9966


In [24]:
from pyspark.sql.functions import expr, col

In [25]:
# what percent of fire dept calls are on holidays?
(calls
 .join(holidays, on=calls["Call_Date"] == holidays["date"], how="left")
 .agg(expr("COUNT(holiday) / COUNT(*)"))
 .toPandas())

,(COUNT(holiday) / COUNT(1))
0,0.015863


In [26]:
import requests

# https://spark.apache.org/docs/latest/monitoring.html#rest-api
# http://localhost:4040/api/v1/applications
# http://localhost:4040/api/v1/applications/{app_id}/executors
# look for "totalTasks"

r = requests.get("http://localhost:4040/api/v1/applications")
r.raise_for_status()
r.json()

[{'id': 'app-20231030131841-0000',
  'name': 'cs544',
  'attempts': [{'startTime': '2023-10-30T13:18:38.684GMT',
    'endTime': '1969-12-31T23:59:59.999GMT',
    'lastUpdated': '2023-10-30T13:18:38.684GMT',
    'duration': 7478648,
    'sparkUser': 'root',
    'completed': False,
    'appSparkVersion': '3.5.0',
    'startTimeEpoch': 1698671918684,
    'endTimeEpoch': -1,
    'lastUpdatedEpoch': 1698671918684}]}]

In [29]:
app_id = r.json()[0]["id"]
app_id

'app-20231030131841-0000'

In [31]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

[0, 41, 33]

In [36]:
df = spark.table("calls").sample(True, 0.01).repartition(1).cache()

In [37]:
df.count()

59768

In [38]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]

[0, 44, 38]

In [39]:
for i in range(30):
    df.count()

In [40]:
r = requests.get(f"http://localhost:4040/api/v1/applications/{app_id}/executors")
r.raise_for_status()
[exec["totalTasks"] for exec in r.json()]
# cache with MEMORY_2, then they both would have had some work

[0, 44, 68]

# Concept of Hash Partitioning

In [41]:
# hash function:
# takes anything (e.g., just some bytes containing whatever)
# returns a number (deteriministic, but ideally not with an obvious pattern)

In [44]:
print(hash(b"a"))
print(hash(b"b"))
print(hash(b"c"))
print(hash(b"d"))
print(hash(b"e"))
print(hash(b"f"))

4048120696332817282
-7032740829281490393
-3557164702627286655
4365002733845661407
251757472463859401
-8027134606503610815


In [45]:
# use this to deterimine what hash partition a value goes in
print(hash(b"a") % 5)
print(hash(b"b") % 5)
print(hash(b"c") % 5)
print(hash(b"d") % 5)
print(hash(b"e") % 5)
print(hash(b"f") % 5)

2
2
0
2
1
0


In [47]:
partitions = [[], [], [], [], []]
for letter in "aaaabbbefghihijkllmlm":
    partition_idx = hash(letter) % len(partitions)
    partitions[partition_idx].append(letter)
partitions

[['f'],
 ['e', 'g', 'l', 'l', 'm', 'l', 'm'],
 ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'i', 'i'],
 ['h', 'h', 'k'],
 ['j']]

In [51]:
spark.sql("""
SELECT Call_Type, COUNT(*) as count
FROM calls
GROUP BY Call_Type
""").explain("formatted")

== Physical Plan ==
AdaptiveSparkPlan (5)
+- HashAggregate (4)
   +- Exchange (3)
      +- HashAggregate (2)
         +- Scan parquet  (1)


(1) Scan parquet 
Output [1]: [Call_Type#231]
Batched: true
Location: InMemoryFileIndex [hdfs://nn:9000/sf.parquet]
ReadSchema: struct<Call_Type:string>

(2) HashAggregate
Input [1]: [Call_Type#231]
Keys [1]: [Call_Type#231]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#24645L]
Results [2]: [Call_Type#231, count#24646L]

(3) Exchange
Input [2]: [Call_Type#231, count#24646L]
Arguments: hashpartitioning(Call_Type#231, 200), ENSURE_REQUIREMENTS, [plan_id=1689]

(4) HashAggregate
Input [2]: [Call_Type#231, count#24646L]
Keys [1]: [Call_Type#231]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#24642L]
Results [2]: [Call_Type#231, count(1)#24642L AS count#24641L]

(5) AdaptiveSparkPlan
Output [2]: [Call_Type#231, count#24641L]
Arguments: isFinalPlan=false


